In [141]:
import os
import ee
import geemap
import pandas as pd

Upgrade geemap to the latest version

In [142]:
# geemap.update_package()

In [143]:
Map_S5P = geemap.Map()
Map_S5P

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [144]:
# Create a common region of interest
roi = ee.Geometry.Polygon([[[-121.39140625, 35.650331512181495],
          [-121.39140625, 30.687240701629882],
          [-112.7341796875, 30.687240701629882],
          [-112.7341796875, 35.650331512181495]]])

In [145]:
# Data COllection for July
image_S5P = ee.ImageCollection("COPERNICUS/S5P/NRTI/L3_NO2") \
    .filterBounds(roi) \
    .filterDate('2018-07-01', '2018-07-31') \
    .sort('ALGORITHM_VERSION') \
    .select('NO2_column_number_density') \
    .median() \
    .clip(roi)

vis_params_S5P = {
    'min': 0,
    'max': 0.0002,
    'bands': ['NO2_column_number_density'],
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

Map_S5P.addLayer(image_S5P, vis_params_S5P, "Sentinel-5P TROPOMI NO2")


image_LANDSAT8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2018-07-01', '2018-07-31') \
    .sort('CLOUD_COVER') \
    .select('B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11', 'sr_aerosol') \
    .median() \
    .clip(roi)

vis_params_L8 = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}


Map_S5P.addLayer(image_LANDSAT8, vis_params_L8, "Landsat-8")

In [146]:
ground_truth= image_LANDSAT8.addBands(image_S5P)

Download sample data

In [149]:
work_dir = os.path.expanduser('/Users/mdnhuda/Desktop/PARC IEM Data/Correlation for Month with Scatter')
in_shp = os.path.join(work_dir, 'geometry_shape.shp')



In [150]:
in_fc = geemap.shp_to_ee(in_shp)
Map.addLayer(in_fc, {}, 'Shape FIle Geometry')

Export pixel values as a shapefile

Export pixel values as a csv

In [151]:
proj = ee.Projection('EPSG:4326')
reproj_image = ground_truth.resample('bilinear').reproject(crs=proj, scale=30)


In [152]:
out_csv = os.path.join(work_dir, 'ground_truth.csv')
geemap.extract_values_to_points(in_fc, reproj_image, out_csv)

Generating URL ...
Please wait ...
Data downloaded to /Users/mdnhuda/Desktop/PARC IEM Data/Correlation for Month with Scatter/ground_truth.csv


In [153]:
df=pd.read_csv('ground_truth.csv')

In [154]:
# use pd.concat to join the new columns with your original dataframe
df = pd.concat([df,pd.get_dummies(df['Class'], prefix='Class')],axis=1)

# now drop the original 'country' column (you don't need it anymore)
df.drop(['Class'],axis=1, inplace=True)

In [155]:
df

,Lon,Lat,B10,B11,B1,B2,B3,B4,B5,NO2_column_number_density,...,Class_forest,Class_grass,Class_highway,Class_industry,Class_land,Class_nodevelop,Class_ocean,Class_open_space,Class_soil,Class_water
0,-115.972790,33.443743,3088.872633,3031.251681,951.070003,1163.310100,1646.042020,2082.418453,2708.291787,0.000080,...,0,0,0,0,0,0,0,0,0,1
1,-115.884899,33.333665,3078.078290,3016.259870,1059.859170,1270.228582,1749.907874,2188.771126,2793.799268,0.000081,...,0,0,0,0,0,0,0,0,0,1
2,-115.725597,33.292351,3073.652689,3006.936830,1201.703996,1415.354165,1907.518075,2376.821580,2931.783636,0.000081,...,0,0,0,0,0,0,0,0,0,1
3,-115.780529,33.241828,3067.443989,3001.036418,1196.500188,1405.098538,1884.736909,2331.968599,2908.569845,0.000081,...,0,0,0,0,0,0,0,0,0,1
4,-115.725597,33.172887,3058.445377,2989.031236,1294.809280,1500.481961,1976.908437,2424.149999,2986.958621,0.000082,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,-116.053294,34.539297,3154.656698,3094.634338,1273.645942,1528.579284,2091.808409,2553.207292,2955.531310,0.000079,...,0,0,0,0,0,0,0,0,1,0
179,-116.040934,34.552305,3154.281705,3094.245257,1280.961330,1536.447702,2102.622649,2568.313611,2976.415604,0.000079,...,0,0,0,0,0,0,0,0,1,0
180,-116.102045,34.642175,3154.380325,3096.573065,1236.180748,1489.423349,2051.418327,2503.179952,2920.237726,0.000079,...,0,0,0,0,0,0,0,0,1,0
181,-115.806788,34.552305,3150.141946,3086.301870,1441.793193,1708.201615,2316.071758,2854.919206,3313.487237,0.000079,...,0,0,0,0,0,0,0,0,1,0
